# Demo Evaluate New Games and/or Characters

**Purpose**: The purpose of this notebook is to provide an example/demo/copy-pasteable template for running analysis using the simulation engine.


## Setup

These instruction assume you are doing in this using the devcontainer extention in VSCode (if you haven't done that check the Contributing.md file in the main repo for instructions).

In [1]:
# Simulation engine core modules
from dcs_simulation_engine.core.run_manager import RunManager
from dcs_simulation_engine.core.game_config import GameConfig
import dcs_simulation_engine.helpers.database_helpers as dbh

In [2]:
# Load the json with the new character definition
import json

file_path = "new-character.json"
with open(file_path, "r", encoding="utf-8") as f:
    new_character_dict = json.load(f)

print(new_character_dict)

{'hid': 'human-new', 'short_description': 'A new human.', 'inclusion_reason': 'To represent ....', 'common_descriptors': ['human', 'biological', 'visually-impaired', 'human-like-cognition'], 'anthronormal_divergence': {'Sensory-divergence': 'Reduced visual acuity, field, or clarity.'}, 'interaction_examples': None, 'long_description': 'I can perceive light, shapes, motion, and sometimes text or detail, but my vision is limited compared to typical humans. Depending on the cause, I may have blurred central vision, reduced peripheral vision, low contrast sensitivity, impaired depth perception, or difficulty detecting fine detail. My visual capacity might fluctuate depending on lighting, fatigue, or assistive tools.\n\nI often rely on enlarged text, high-contrast visuals, audio descriptions, tactile feedback, or assistive technologies such as screen readers or magnifiers. In unfamiliar or visually complex environments, I use memory, mobility aids (like canes), or guidance from others. My c

In [3]:
# Make sure we use a mock database for running tests
from helpers import use_mongomock_for_db

use_mongomock_for_db(seed_data=True)

2025-11-07 14:26:35.097 | DEBUG    | helpers:_seed_from_dir:70 - Seeding 9 docs into collection 'characters' from characters.json to run tests.
2025-11-07 14:26:35.099 | DEBUG    | helpers:_seed_from_dir:66 - No documents found in players.json; skipping.
2025-11-07 14:26:35.099 | DEBUG    | helpers:_seed_from_dir:66 - No documents found in runs.json; skipping.


In [4]:
# Get the database and make sure its mongomock (not real)
db = dbh.get_db()

import mongomock

assert isinstance(db.client, mongomock.MongoClient)

In [5]:
# List character hids from the database
hids = [doc.get("hid") for doc in db["characters"].find({}, {"hid": 1, "_id": 0})]
hids

['human-normative',
 'human-low-vision',
 'human-non-hearing',
 'human-non-ambulatory',
 'human-multi-divergent-complex',
 'llm-gpt5',
 'flatworm',
 'thermostat',
 'algorithm-sort']

In [6]:
# Add your new character
db["characters"].insert_one(new_character_dict)
# List character hids again to see the new one added
hids = [doc.get("hid") for doc in db["characters"].find({}, {"hid": 1, "_id": 0})]
hids

['human-normative',
 'human-low-vision',
 'human-non-hearing',
 'human-non-ambulatory',
 'human-multi-divergent-complex',
 'llm-gpt5',
 'flatworm',
 'thermostat',
 'algorithm-sort',
 'human-new']

In [7]:
# Make sure your new game is valid
from pathlib import Path

new_game_config = GameConfig.from_yaml(Path("new-game.yml"))

In [8]:
# Init the simulation with your new game
run = RunManager.create(
    game=new_game_config,
    source="manual-test",
    pc_choice="human-normative",
    npc_choice="human-new",
)

2025-11-07 14:26:38.781 | DEBUG    | dcs_simulation_engine.core.run_manager:create:147 - Create class method called with game=name='New Game Demo' description='A demo\n' version='1.0.0' authors=['DCS'] stopping_conditions={'runtime_seconds': ['>=300'], 'turns': ['>=10']} state_overrides={} access_settings=AccessSettings(user=ValiditySelector(valid={'players': {}}, invalid={}), consent_form={}) data_collection_settings={'save_runs': False} character_settings=CharacterSettings(pc=ValiditySelector(valid={'characters': {'where': {'hid': 'human-normative'}}}, invalid={}), npc=ValiditySelector(valid={'characters': {'where': {'hid': 'human-new'}}}, invalid={})) graph_config=GraphConfig(name='demo-game-graph', description='A graph that defines the flow of the demo game.', state_overrides=None, nodes=[Node(name='command_filter', kind='builtin.command_filter', kwargs={'command_handlers': {'help': {'special_user_message': {'type': 'info', 'content': 'Describe an action...\n- Eg. If your character

## Run the simulation step my step or to defined completion

### Step by step run

In [9]:
# step through the simulation one step at a time
run.step()

2025-11-07 14:26:49.872 | DEBUG    | dcs_simulation_engine.core.run_manager:_ensure_stopping_conditions:498 - Checking stopping conditions...
2025-11-07 14:26:49.874 | DEBUG    | dcs_simulation_engine.core.simulation_graph.core:invoke:87 - Resetting fields before invoking graph: special_user_message
2025-11-07 14:26:49.875 | DEBUG    | dcs_simulation_engine.core.simulation_graph.core:invoke:90 - Invoking SimulationGraph...
2025-11-07 14:26:49.876 | DEBUG    | dcs_simulation_engine.core.simulation_graph.core:invoke:91 - Input keys: dict_keys(['events', 'lifecycle', 'exit_reason', 'special_user_message', 'event_draft', 'invalid_reason', 'retries', 'retry_limits', 'forms'])
2025-11-07 14:26:49.879 | DEBUG    | dcs_simulation_engine.core.simulation_graph.conditions:eval_condition:51 - Condition eval False for state['special_user_message']
2025-11-07 14:26:49.882 | DEBUG    | dcs_simulation_engine.core.simulation_graph.conditions:eval_condition:51 - Condition eval True for state['lifecycle'

{'events': [AIMessage(content='You enter a new space. In this space, a quiet community reading room lit by soft daylight through high windows: you stand by the doorway, and across a round table a person with a cane and a magnifier sits beside a stack of large-print books, tapping a finger on the table as they listen.', additional_kwargs={}, response_metadata={}, id='d291f07b-9882-4653-888e-0c6be0e5cd1a')],
 'lifecycle': 'UPDATE',
 'exit_reason': '',
 'special_user_message': {'type': 'info',
  'content': '# Welcome\nIn this game there is no predefined objective or task. You can just engage freely with the other character. \n\nYou are playing as: human-normative (A human with standard normal form, function, goals, sensory, perceptual, regulatory and action modalities.)\n\nThe simulator is playing as: human-new (A new human.)'},
 'event_draft': {'type': 'ai',
  'content': 'You enter a new space. In this space, a quiet community reading room lit by soft daylight through high windows: you s

####  "Play to completion" using GUI

In [ ]:
from helpers import play_gradio

run.play(input_provider=play_gradio)